In [ ]:
!apt install ffmpeg
!pip install spleeter
!pip install librosa
!pip install pydub
!pip install SpeechRecognition
import speech_recognition as sr
from pydub import AudioSegment
import numpy as np
import csv
import librosa

In [ ]:
!spleeter separate /content/vocals.wav-o output/ # dont run

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the audio file (replace 'audio_file_path' with the actual path)
audio_file_path = '/content/drive/MyDrive/Project Sample audio/Aj Jane.wav'
audio = AudioSegment.from_file(audio_file_path, format="wav")

In [ ]:
import speech_recognition as sr
# Perform speech recognition using the SpeechRecognition library
recognizer = sr.Recognizer()
with sr.AudioFile(audio_file_path) as source:
    audio_data = recognizer.record(source)

text = recognizer.recognize_google(audio_data, language='hi-IN')

In [ ]:
# Split the recognized text into words
words = text.split()

num_words=len(words) # having the number of data


In [ ]:
# Calculate word timestamps as before
word_duration = len(audio) / len(words)
timestamps = [] #list
for i, word in enumerate(words):
    start_time = i * word_duration
    end_time = (i + 1) * word_duration
    timestamps.append((word, start_time, end_time))

# timestamps(words, start_time, end_time)

In [ ]:
print(timestamps)

In [ ]:
# Calculate the fundamental frequency (pitch) of each word
word_pitch = {}
for word, start, end in timestamps:
    word_audio = audio[int(start):int(end)]
    y, sr = librosa.load(word_audio.export(format="wav"), sr=None)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    # Extract the pitch (frequency) information
    pitch = np.mean(pitches)
    word_pitch[word] = pitch

In [ ]:
word_pitch

In [ ]:

# Convert pitch (frequency) into corresponding musical notes
def frequency_to_note_name(frequency):
    if frequency <= 0:
        return "Unknown"
    n = 12 * (np.log2(frequency / 440)) + 49
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    #octave = int(n // 12)
    note = int(n % 12)
    return f"{note_names[note]}"#{octave}"



In [ ]:
#difference between notes

def note_diff(initial_index,final_index):
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    initial_index = note_names.index(initial_index)
    final_index = note_names.index(final_index)
    if final_index >= initial_index:
        distance = final_index - initial_index
    else:
        distance = len(note_names) + final_index - initial_index
    return distance


In [ ]:
headers = ["Pid", "Words", "s_time", "e_time", "time", "Note", "Note_diff"]

In [ ]:
# Print estimated timestamps, exact frequency, and corresponding musical notes
print("Estimated Word Timestamps, Frequencies, and Musical Notes:")
previous_note = None  # Initialize the previous note as None
for word, start, end in timestamps:
    pitch = word_pitch[word]
    note = frequency_to_note_name(pitch)
    diff = note_diff(previous_note, note) if previous_note else 0  # Calculate note difference
    previous_note = note  # Update the previous note
    print(f"Word: {word} | Start Time: {start} ms | End Time: {end} ms | Pitch (Frequency): {pitch:.2f} Hz | Note: {note} | Note_diff: {diff}")

In [ ]:
# Write data to the CSV file
with open("musical_data.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()

    pid = 1  # Initialize a PID (Process ID) counter
    previous_note = None  # Initialize the previous note as None
    for word, start, end in timestamps:
        pitch = word_pitch[word]
        note = frequency_to_note_name(pitch)
        diff = note_diff(previous_note, note) if previous_note else 0  # Calculate note difference
        previous_note = note  # Update the previous note
        writer.writerow({
            "Pid": pid,
            "Words": word,
            "s_time": start,
            "e_time": end,
            "time": end - start,
            "Note": note,
            "Note_diff": diff
        })

        pid += 1

In [ ]:
import pandas as pd

csvFile = pd.read_csv('musical_data.csv')

In [ ]:
print(csvfile)

In [ ]:
from google.colab import files

files.download('musical_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>